In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 72.2 MB/s eta 0:00:00


In [4]:
pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.6/166.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 4.9 MB/s eta 0:00:

In [5]:
pip install sentence_transformers

In [8]:
!pip install --upgrade chromadb pdfplumber sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.2.1
    Uninstalling sentence-transformers-3.2.1:
      Successfully uninstalled sentence-transformers-3.2.1


In [4]:
from chromadb import PersistentClient
from sentence_transformers import SentenceTransformer
import pdfplumber

# Initialize ChromaDB client
db_path = "/content/drive/My Drive/machine_learning_lab/chroma_db"
chroma_client = PersistentClient(path=db_path)
# Create or get a collection in ChromaDB
collection_name = "pdf-semi-structured-data1"
# Check if the collection already exists
if collection_name not in chroma_client.list_collections():
    # Create the collection if it doesn't exist
    collection = chroma_client.create_collection(name=collection_name)
else:
    # Get the existing collection
    collection = chroma_client.get_collection(name=collection_name)


# Load pre-trained embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

def extract_and_chunk_pdf(file_path):
    """
    Extract and chunk data from a PDF.
    Args:
        file_path: Path to the PDF file.
    Returns:
        chunks: List of extracted chunks (text, tables, graph descriptions).
    """
    chunks = []

    with pdfplumber.open(file_path) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):
            # Extract text content
            text = page.extract_text()
            if text:
                # Split text into paragraphs or sentences
                paragraphs = text.split("\n")
                for para in paragraphs:
                    chunks.append({
                        "content": para.strip(),
                        "type": "text",
                        "source": f"Page {page_num}"
                    })

            # Extract table data
            tables = page.extract_tables()
            for table_index, table in enumerate(tables):
                table_content = "\n".join([", ".join([str(cell) if cell is not None else "" for cell in row]) for row in table])
                chunks.append({
                    "content": table_content.strip(),
                    "type": "table",
                    "source": f"Page {page_num} Table {table_index+1}"
                })

    return chunks

def embed_and_store_chunks(chunks, collection):
    """
    Embed and store chunks into ChromaDB.
    Args:
        chunks: List of chunks to embed.
        collection: ChromaDB collection object.
    """
    documents = [chunk["content"] for chunk in chunks]
    embeddings = model.encode(documents).tolist()
    ids = [f"chunk-{i}" for i in range(len(chunks))]
    metadatas = [{"type": chunk["type"], "source": chunk["source"]} for chunk in chunks]

    collection.add(ids=ids, embeddings=embeddings, metadatas=metadatas, documents=documents)

def query_pdf(query, collection):
    """
    Query ChromaDB for the most relevant chunks.
    Args:
        query: User's query string.
        collection: ChromaDB collection object.
    Returns:
        Top chunks retrieved.
    """
    query_embedding = model.encode([query]).tolist()[0]
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=3  # Retrieve top 3 relevant chunks
    )
    return results


# Example: Process PDF and store data
pdf_path = "gdrive/My Drive/machine_learning_lab/Example.pdf"  # Replace with your PDF path
chunks = extract_and_chunk_pdf(pdf_path)
print(f"Extracted {len(chunks)} chunks.")

# Embed and store chunks in ChromaDB
embed_and_store_chunks(chunks, collection)
print("Chunks stored in ChromaDB.")

# Example: Query the stored data
user_query = "What is the unemployment rate for bachelor's degrees?"
results = query_pdf(user_query, collection)

print("\nQuery Results:")
for doc, meta, score in zip(results["documents"], results["metadatas"], results["distances"]):
    print(f"Document: {doc}, Metadata: {meta}, Score: {score}")


Extracted 180 chunks.
Chunks stored in ChromaDB.

Query Results:
Document: ['Source: U.S. Bureau of Labor Statistics', 'class received grades of 72 or', '\uf075 The chart indicates that 15% of the income is spent on'], Metadata: [{'source': 'Page 6', 'type': 'text'}, {'source': 'Page 18', 'type': 'text'}, {'source': 'Page 10', 'type': 'text'}], Score: [0.9479191899299622, 1.4064095286106586, 1.4349067211151123]


In [5]:
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}:")
    print(chunk)
    print("-" * 50)  # Separator for readability


Chunk 1:
{'content': 'Tables, Charts, and', 'type': 'text', 'source': 'Page 1'}
--------------------------------------------------
Chunk 2:
{'content': 'Graphs', 'type': 'text', 'source': 'Page 1'}
--------------------------------------------------
Chunk 3:
{'content': 'with Examples from History, Economics,', 'type': 'text', 'source': 'Page 1'}
--------------------------------------------------
Chunk 4:
{'content': 'Education, Psychology, Urban Affairs and', 'type': 'text', 'source': 'Page 1'}
--------------------------------------------------
Chunk 5:
{'content': 'Everyday Life', 'type': 'text', 'source': 'Page 1'}
--------------------------------------------------
Chunk 6:
{'content': 'REVISED: MICHAEL LOLKUS 2018', 'type': 'text', 'source': 'Page 1'}
--------------------------------------------------
Chunk 7:
{'content': 'Tables, Charts, and', 'type': 'text', 'source': 'Page 3'}
--------------------------------------------------
Chunk 8:
{'content': 'Graphs Basics', 'type': 'text',

In [11]:
pip install "camelot-py[cv]"  # Install the correct library


INFO: pip is looking at multiple versions of camelot-py[cv] to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.9 MB/s eta 0:00:00


In [2]:
pip install opencv-python-headless

In [24]:
import pdfplumber

with pdfplumber.open("gdrive/My Drive/machine_learning_lab/Example.pdf") as pdf:
    page = pdf.pages[5]  # Page 6 is index 5
    table = page.extract_table()
    print(table)


[['Year', '2010', '2011', '2012', '2013', '2014', '2015'], ['All Industries', '26093515', '27535971', '28663246', '29601191', '30895407', '31397023'], ['Manufacturing', '4992521', '5581942', '5841608', '5953299', '6047477', '5829554'], ['Finance,\nInsurance, Real\nEstate, Rental,\nLeasing', '4522451', '4618678', '4797313', '5031881', '5339678', '5597018'], ['Arts,\nEntertainment,\nRecreation,\nAccommodation,\nand Food Service', '964032', '1015238', '1076249', '1120496', '1189646', '1283813'], ['Other', '15614511', '16320113', '16948076', '17495515', '18318606', '18686638']]


In [6]:
pip install fitz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.8 MB/s eta 0:00:00


In [4]:
pip install frontend

In [7]:

pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 25.5 MB/s eta 0:00:00


In [9]:
import fitz  # PyMuPDF
pdf_path="gdrive/My Drive/machine_learning_lab/Example.pdf"
# Open PDF and select page
pdf_document = fitz.open(pdf_path)
page_number = 2
page = pdf_document[page_number - 1]  # Page index starts at 0

# Render page as an image
pix = page.get_pixmap()
output_image_path = "gdrive/My Drive/machine_learning_lab/graph_page2.png"
pix.save(output_image_path)
print(f"Graph saved as image: {output_image_path}")

Graph saved as image: gdrive/My Drive/machine_learning_lab/graph_page2.png


In [11]:
pip install pytesseract

In [19]:
!sudo apt-get update
!sudo apt-get install -y tesseract-ocr
!sudo apt-get install -y libtesseract-dev


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [60.9 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [3,550 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 P

In [20]:
import pytesseract

# Set the path for tesseract
pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"


In [28]:
from PIL import Image
import pytesseract

# Path to your image
image_path = "gdrive/My Drive/machine_learning_lab/graph_page2.png"  # Replace with your image path

# Open the image using PIL
img = Image.open(image_path)

# Extract text from the image
text = pytesseract.image_to_string(img)
print(text)
graph_json = {"graph_data": text}
print(graph_json)

Earnings and Unemployment Rates by Educational Attainment

Unemployment rate in 2013 (%) Median weekly earnings in 2013 ($)

Doctoral degree
Professional degree
Master's degree
Bachelor's degree
Associate's degree
Some college, no degree
High school diploma

Alll workers: 6.1% Alll workers: $827
Source: Bureau of Labor Statistics, 2014.

 

{'graph_data': "Earnings and Unemployment Rates by Educational Attainment\n\nUnemployment rate in 2013 (%) Median weekly earnings in 2013 ($)\n\nDoctoral degree\nProfessional degree\nMaster's degree\nBachelor's degree\nAssociate's degree\nSome college, no degree\nHigh school diploma\n\nAlll workers: 6.1% Alll workers: $827\nSource: Bureau of Labor Statistics, 2014.\n\n \n\x0c"}


In [31]:
import json

# Example: Combine outputs
final_output = {
    "tables": table, #json.loads(table[0].df.to_json(orient='records')),
    "graphs": graph_json
}

# Save to a file
with open("output.json", "w") as f:
    json.dump(final_output, f, indent=4)

print("Final data saved to output.json")
print(final_output)

Final data saved to output.json
{'tables': [['Year', '2010', '2011', '2012', '2013', '2014', '2015'], ['All Industries', '26093515', '27535971', '28663246', '29601191', '30895407', '31397023'], ['Manufacturing', '4992521', '5581942', '5841608', '5953299', '6047477', '5829554'], ['Finance,\nInsurance, Real\nEstate, Rental,\nLeasing', '4522451', '4618678', '4797313', '5031881', '5339678', '5597018'], ['Arts,\nEntertainment,\nRecreation,\nAccommodation,\nand Food Service', '964032', '1015238', '1076249', '1120496', '1189646', '1283813'], ['Other', '15614511', '16320113', '16948076', '17495515', '18318606', '18686638']], 'graphs': {'graph_data': "Earnings and Unemployment Rates by Educational Attainment\n\nUnemployment rate in 2013 (%) Median weekly earnings in 2013 ($)\n\nDoctoral degree\nProfessional degree\nMaster's degree\nBachelor's degree\nAssociate's degree\nSome college, no degree\nHigh school diploma\n\nAlll workers: 6.1% Alll workers: $827\nSource: Bureau of Labor Statistics, 201